In [1]:
import schedule
import time
from gtts import gTTS
import os
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')

In [2]:
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/MARVIN1.mp3')
import winsound
import numpy as np
import pandas as pd
import math
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
from datetime import datetime as dt
#import socketio

In [3]:
#import requests
import json
from scipy.signal import argrelextrema
import pytz
import itertools
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
#import fxcmpy

In [4]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [5]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [6]:
##  INITIAL SETUP



## WATCHLIST = ['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'EUR/USD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'USD/CAD', 'USD/CHF', 'USD/JPY', 'XAG/USD', 'XAU/USD', 'BCH/USD', 'BTC/USD', 'EOS/USD', 'ETH/USD', 'LTC/USD', 'XLM/USD', 'XRP/USD']

## FOREX = ## WATCHLIST = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

#TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
#con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')

#ZA = []
#ZB = []
#ZC = []
#TRADES = []
pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

def brutus():
    
#    with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/BRUTUS_Mark_II/Levels.xlsx') as writerdft:
#        with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/BRUTUS_Mark_II/Results.xlsx') as writerdfd:

    h = len(pair)

    #winsound.PlaySound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/runningwav.wav', winsound.SND_ASYNC | winsound.SND_ALIAS )

    column_names = ["pairing","position","entry","tkprofit","stloss"]

    purps = pd.DataFrame(columns = column_names)

    for x in pair:



## LEVELS ANALYSIS  -  GOING BACK 10 YEARS WITH ONE-WEEK CANDLESTICKS



        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)

        S = pd.DataFrame(Srates)
        S['time']=pd.to_datetime(S['time'], unit='s')
        S = S.drop(columns=['tick_volume', 'spread', 'real_volume'])
        S = S.rename({'time': 'date', 'open': 'a', 'high': 'b', 'low': 'c', 'close': 'd'}, axis='columns')
        S = S.set_index('date')
        S = S.assign(e=S['a'])
        S = S.assign(f=S['b'])
        S = S.assign(g=S['c'])
        S = S.assign(h=S['d'])
        S = S.assign(tickqty=0)
        S = S.assign(open=S['a'])
        S = S.assign(close=S['d'])
        S = S.assign(high=S['b'])
        S = S.assign(low=S['c'])
        S = S.tz_localize('Etc/GMT-7')
        S = S.tz_convert('UTC')



        S = S.assign(MOVE=(S['close'] - S['open']))
        SAvgCandle = np.average(abs(S['MOVE']))
        STDCandle = np.std(abs(S['MOVE']))
        CP = S['close'].iloc[-1]
        scope = SAvgCandle*15
        CPU = CP + scope
        CPD = CP - scope
        sskew = 1
        SCM = SAvgCandle*sskew                                 ## THIS IS THE % MARGIN FOR THE CONFLUENCE OF LEVELS

        CNDL = (S.iloc[:, 10]) - (S.iloc[:, 9])
        S['CNDL'] = np.where(CNDL > 0, 1, -1)
        S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
        S = S[S.INFL != 0]
        S['INFL'] = S['CNDL']*-1
        S['LVL'] = np.where(S.INFL == 1, S.high, S.low)
        S = S.assign(SIZE=abs(S['LVL'] - (S['LVL'].shift(-1))))

        A = np.array(S.LVL)

        B = list(combinations(A, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])

        T = list(zip(Ba,Bb))

        table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
        table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
        table = table.sort_values(['a'], ascending=False)
        table = table.rename(columns={'b': 'Pings'})
        #table = table[table.index <= CPU]
        #table = table[table.index >= CPD]
        table = table.fillna(0)
        table = table[table['max'] != 0]
        pingsavg = np.average(table['Pings'])
        table = table.assign(girth=table['Pings']/pingsavg)
        table = table.assign(level=table.index)

        levels = list(table['level'])
        levsize = list(table['girth'])

        LEVELS = zip(levels,levsize)
        LEVELS = pd.DataFrame(LEVELS)





## NOW WE WILL SET UP A BRAND NEW TABLE OF DATA WHICH WILL BE OUR MAIN RESULTS TABLE


## THE BEGINNING OF THE MAIN RESULTS TABLE  -  GOING BACK 25 DAYS WITH FOUR-HOUR CANDLESTICKS



        Drates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 600)


        D = pd.DataFrame(Drates)
        D['time']=pd.to_datetime(D['time'], unit='s')
        D = D.drop(columns=['tick_volume', 'spread', 'real_volume'])
        D = D.rename({'time': 'date', 'open': 'a', 'high': 'b', 'low': 'c', 'close': 'd'}, axis='columns')
        D = D.set_index('date')
        D = D.assign(e=D['a'])
        D = D.assign(f=D['b'])
        D = D.assign(g=D['c'])
        D = D.assign(h=D['d'])
        D = D.assign(tickqty=0)
        D = D.assign(open=D['a'])
        D = D.assign(close=D['d'])
        D = D.assign(high=D['b'])
        D = D.assign(low=D['c'])
        D = D.tz_localize('Etc/GMT-7')
        D = D.tz_convert('UTC')

        D = D.assign(MOVE=(D['close'] - D['open']))
        DAvgCandle = np.average(abs(D['MOVE']))
        skew = 1
        CM = DAvgCandle*skew
        l = int(D['close'].count())
        lin = np.linspace(1,l,num=l)
        zeros = [0]*l
        zeros = np.transpose(zeros)


## HERE'S WHERE WE IDENTIFY WHAT EACH CANDLE IS SHAPED LIKE AND ITS POSITION ON THE WICK


        D = D.assign(STICK=(D['high'] - D['low']))
        D = D.assign(PSTICK=abs(D['MOVE']/D['STICK']))
        D['DIRECT'] = np.where(D.MOVE > 0, 'Bullish', 'Bearish')

        ANIMAL = []
        i = 0
        for t in lin: 
            if D.iloc[i,15] <= 1.1:
                if D.iloc[i,15] < 0.85:
                    if D.iloc[i,15] < 0.65:
                        if D.iloc[i,15] < 0.50:
                            if D.iloc[i,15] <= 0.30:
                                if D.iloc[i,15] <= 0.15:
                                    animal = 'Doji'
                                else:
                                    animal = 'Frog'
                            else:
                                animal = 'Cat'
                        else:
                            animal = 'Dog'
                    else:
                        animal = 'Horse'
                else:
                    animal = 'Shark'

            ANIMAL.append(animal)
            i = i + 1
        D['ANIMAL'] = ANIMAL

        GAUGE = []
        i = 0
        for t in lin: 

            cndmid = (D.iloc[i,9]+D.iloc[i,10])/2
            stickmid = (D.iloc[i,11]+D.iloc[i,12])/2

            if D.iloc[i,14] == 0:
                midpoint = D.iloc[i,9]
            else:
                midpoint = (cndmid - stickmid)/D.iloc[i,14]

            if midpoint >= 0.075:
                if midpoint >= 0.175:
                    if midpoint >= 0.25:
                        gauge = 'Top Saturated'
                    else:
                        gauge = 'Top Stacked'
                else:
                    gauge = 'Top Semi-Stacked'
            elif midpoint <= -0.075:
                if midpoint <= -0.175:
                    if midpoint <= -0.25:
                        gauge = 'Bottom Saturated'
                    else:
                        gauge = 'Bottom Stacked'
                else:
                    gauge = 'Bottom Semi-Stacked'
            else:
                gauge = 'Centered'

            GAUGE.append(gauge)
            i = i + 1
        D['GAUGE'] = GAUGE


## NOW WE'RE GOING TO ADD A COLUMN FOR THE INSTANTANEOUS TREND

## THIS IS CALCULATED BY SIMPLY TAKING THE AVERAGE OF THE MOVE COLUMN OVER THE INTERVAL SPECIFIED


        thh = 168                           ## CANDLESTICK INTERVAL FOR NET TREND (SET TO 168 CANDLES NOW)
        tl = l - thh
        tlin = np.linspace(1,tl,num=tl)
        tlinb = np.linspace(1,thh,num=thh)
        TREND = []
        for t in tlinb:
            TREND.append(0)
        i = thh                                                                                       
        for t in tlin:
            ii = i - thh
            movetrend = np.sum(list(D.iloc[ii:i,13]))
            i = i + 1
            TREND.append(movetrend)
        D['TREND'] = TREND



## SINGLE-CANDLESTICK FORMATIONS



        READING=[]
        i = 0
        for t in lin: 
            if D.iloc[i,19] < 0:
                if D.iloc[i,18] == 'Top Saturated':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'HAMMER'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Top Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'MALLET'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Top Semi-Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Centered':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Bottom Semi-Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Bottom Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Bottom Saturated':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,17]
                else:
                    Read = D.iloc[i,17]
            elif D.iloc[i,19] > 0:
                if D.iloc[i,18] == 'Bottom Saturated':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'SHOOTING STAR'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Bottom Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'ASTEROID'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Bottom Semi-Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Centered':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Top Semi-Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Top Stacked':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,17]
                elif D.iloc[i,18] == 'Top Saturated':
                    if D.iloc[i,17] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,17] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,17] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,17] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,17] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,17] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,17]
                else:
                    Read = D.iloc[i,17]
            else:
                Read = D.iloc[i,17]
            i = i + 1
            READING.append(Read)
        D['READING'] = READING



## DOUBLE-CANDLESTICK FORMATIONS  



        READING2=[]
        READING2.append(0)
        jl = l - 1
        linjj = np.linspace(1,jl,num=jl)
        i = 1
        for t in linjj: 
            ii = i - 1
            if D.iloc[i,19] < 0:
                if D.iloc[ii,16] == 'Bearish':
                    if D.iloc[i,10] > D.iloc[ii,11]:
                        if D.iloc[ii,20] == 'BRAKE':
                            if D.iloc[i,16] == 'Bullish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,20] == 'GLIDE':
                            if D.iloc[i,16] == 'Bullish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,20] == 'PLUMP DOJI':
                            if D.iloc[i,16] == 'Bullish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,20] == 'LONG-LEG DOJI':
                            if D.iloc[i,16] == 'Bullish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,9] == D.iloc[ii,10]:
                        if D.iloc[i,12] == D.iloc[ii,12]:
                            if D.iloc[i,16] == 'Bullish':
                                if D.iloc[i,10] > D.iloc[ii,9]:
                                    Read2 = 'TWEEZERS BOTTOM'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,10] > (D.iloc[ii,9]+D.iloc[ii,10])/2:
                        if D.iloc[i,9] < D.iloc[ii,10]:
                            if D.iloc[ii,16] == 'Bullish':
                                Read2 = 'PIERCING PATTERN'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            elif D.iloc[i,19] > 0:
                if D.iloc[ii,16] == 'Bullish':
                    if D.iloc[i,10] < D.iloc[ii,12]:
                        if D.iloc[ii,20] == 'BRAKE':
                            if D.iloc[i,16] == 'Bearish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,20] == 'GLIDE':
                            if D.iloc[i,16] == 'Bearish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,20] == 'PLUMP DOJI':
                            if D.iloc[i,16] == 'Bearish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,20] == 'LONG-LEG DOJI':
                            if D.iloc[i,16] == 'Bearish':
                                if D.iloc[i,20] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,20] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,10] == D.iloc[ii,9]:
                    if D.iloc[i,11] == D.iloc[ii,11]:
                        if D.iloc[i,16] == 'Bearish':
                            if D.iloc[i,10] < D.iloc[ii,9]:
                                Read2 = 'TWEEZERS TOP'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,10] < (D.iloc[ii,9]+D.iloc[ii,10])/2:
                    if D.iloc[i,9] > D.iloc[ii,10]:
                        if D.iloc[ii,16] == 'Bearish':
                            Read2 = 'DARK CLOUD COVER'
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            else:
                Read2 = 0
            i = i + 1
            READING2.append(Read2)
        D['READING2'] = READING2

        D['READING3'] = zeros

        D['READING4'] = zeros



## NOW WE'LL MAKE A COLUMN FOR THE CURRENT VALUE OF THE ICHIMOKU

## IF SAID VALUE SHOWS BULLISH OUTPUT IS 1, BEARISH OUTPUT IS -1, AND NEUTRAL OUTPUT IS 0


        p = l
        RED = []
        BLUE = []
        GREEN = []
        CLOUDA = []
        CLOUDB = []

        for t in lin:

            if p < 26:
                BLhigh = 1
                BLlow = 1
            else:
                BL = D[p-26:p]
                BLhigh = np.max(BL['high'])
                BLlow = np.min(BL['low'])
            blueline = (BLhigh + BLlow)/2
            BLUE.append(blueline)


        D['Cloud'] = zeros
        D['PvCloudA'] = zeros
        D['PvCloudB'] = zeros

        BLUE.reverse()

        slblu = np.linspace(1,26,num=26)

        D['RedvBlue'] = zeros
        D['PvBlue'] = zeros
        D['PvGreen'] = zeros

        D['BLUE'] = BLUE
        D = D.assign(BluevLast=(D['BLUE'] - D['BLUE'].shift()))
        nq = 0
        for t in lin:
            if D.iloc[nq,31] > 0:
                D.iloc[nq,31] = 1
            elif D.iloc[nq,31] < 0:
                D.iloc[nq,31] = -1
            else:
                D.iloc[nq,31] = 0
            nq = nq + 1 
        nb = 0
        for n in slblu:
            D.iloc[nb,30] = '--'
            D.iloc[nb,31] = '--'
            nb = nb + 1


        D['RSIR'] = zeros
        D['RSID'] = zeros





## HERE WE PLACE A 1 IN THIS COLUMN IF PRICE HAS CLOSED NEAR A MAJOR LEVEL

## WE USE THE DIFFERENCE BETWEEN THE LEVEL AND PRICE AND COMPARE IT TO THE CONFLUENCE MARGIN  CM  !


        OTESTLVL = []
        OTLSIZE = []
        i = 0
        for t in lin:
            levl = 0
            lsize = 0
            q = 0
            for u in levels:
                if abs(D.iloc[i,9]-u) <= CM:
                    levl = u
                    lsize = table.iloc[q,2]
                q = q + 1
            OTESTLVL.append(levl)
            OTLSIZE.append(lsize)
            i = i + 1
        D['OTESTLVL'] = OTESTLVL
        D['OTLSIZE'] = OTLSIZE


        CTESTLVL = []
        CTLSIZE = []
        i = 0
        for t in lin:
            levl = 0
            lsize = 0
            q = 0
            for u in levels:
                if abs(D.iloc[i,10]-u) <= CM:
                    levl = u
                    lsize = table.iloc[q,2]
                q = q + 1
            CTESTLVL.append(levl)
            CTLSIZE.append(lsize)
            i = i + 1
        D['CTESTLVL'] = CTESTLVL
        D['CTLSIZE'] = CTLSIZE


        BREAKLVL = []
        BRLSIZE = []
        i = 0
        for t in lin:
            levl = 0
            lsize = 0
            q = 0
            for u in levels:
                if D.iloc[i,16] == 'Bullish':
                    if D.iloc[i,10] > (u+CM):
                        if D.iloc[i,9] < u:
                            levl = u
                            lsize = table.iloc[q,2]
                        elif D.iloc[i,34] == u:
                            levl = u
                            lsize = table.iloc[q,2]
                elif D.iloc[i,16] == 'Bearish':
                    if D.iloc[i,10] < (u-CM):
                        if D.iloc[i,9] > u:
                            levl = u
                            lsize = table.iloc[q,2]
                        elif D.iloc[i,34] == u:
                            levl = u
                            lsize = table.iloc[q,2]
                q = q + 1
            BREAKLVL.append(levl)
            BRLSIZE.append(lsize)
            i = i + 1
        D['BREAKLVL'] = BREAKLVL
        D['BRLSIZE'] = BRLSIZE



## NOW LET'S BEGIN TO COMPARE DATA TO THE OVERALL REGRESSION LINE OF BEST FIT FOR THE DATAFRAME WE'RE WORKING WITH


        Frates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M15, 0, 200)

        F = pd.DataFrame(Frates)
        F['time']=pd.to_datetime(F['time'], unit='s')
        F = F.drop(columns=['tick_volume', 'spread', 'real_volume'])
        F = F.rename({'time': 'date', 'open': 'a', 'high': 'b', 'low': 'c', 'close': 'd'}, axis='columns')
        F = F.set_index('date')
        F = F.assign(e=F['a'])
        F = F.assign(f=F['b'])
        F = F.assign(g=F['c'])
        F = F.assign(h=F['d'])
        F = F.assign(tickqty=0)
        F = F.assign(open=F['a'])
        F = F.assign(close=F['d'])
        F = F.assign(high=F['b'])
        F = F.assign(low=F['c'])
        F = F.tz_localize('Etc/GMT-7')
        F = F.tz_convert('UTC')
        fl = F['close'].count()
        x1 = np.linspace(1, fl, num=fl)
        y1 = F.iloc[:,10]
        m = sm.OLS(y1, x1).fit()
        slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
        wholeslope = slope

        DITREND = []
        i = 0
        for t in lin: 
            if wholeslope > 0:
                if D.iloc[i,19] > 0:
                    ditrend = 1
                elif D.iloc[i,19] < 0:
                    ditrend = -1
                else:
                    ditrend = 0
            elif wholeslope < 0:
                if D.iloc[i,19] < 0:
                    ditrend = 1
                elif D.iloc[i,19] > 0:
                    ditrend = -1
                else:
                    ditrend = 0
            else:
                ditrend = 0

            DITREND.append(ditrend)                
            i = i + 1

        D['DITREND'] = DITREND


        DSTREND = []
        i = 0
        for t in lin: 
            if D.iloc[i,16] == 'Bullish':
                if D.iloc[i,19] > 0:
                    dstrend = 1
                elif D.iloc[i,19] < 0:
                    dstrend = -1
                else:
                    dstrend = 0
            elif D.iloc[i,16] == 'Bearish':
                if D.iloc[i,19] < 0:
                    dstrend = 1
                elif D.iloc[i,19] > 0:
                    dstrend = -1
                else:
                    dstrend = 0
            else:
                dstrend = 0

            DSTREND.append(dstrend)                
            i = i + 1

        D['DSTREND'] = DSTREND



## NEXT WE WILL INTERPRET ALL OF THE CANDLESTICK DATA IN JUNCTION WITH REGRESSION DATA ABOVE TO FORM COLOR-CODES



        ypa = ['JACK', 'GLIDE']
        grpa = ['MARIBOZU', 'TUBE', 'BOOST']
        goldpa = ['HAMMER', 'SHOOTING STAR', 'DRAGONFLY DOJI', 'GRAVESTONE DOJI', 'LING-LEG DOJI']
        silvpa = ['PLUMP DOJI', 'MALLET', 'ASTEROID', 'DIPPER', 'SKILLET']
        goldpa2 = ['BULLISH ENGULFING', 'BEARISH ENGULFING', 'TWEEZERS TOP', 'TWEEZERS BOTTOM', 'DARK CLOUD COVER', 'PIERCING PATTERN']
        COLOR = []
        i = 0
        for t in lin:
            color = 'white'
            if D.iloc[i,38] != 0:
                if D.iloc[i,36] == 0:
                    if D.iloc[i,23] == 0:
                        if D.iloc[i,22] == 0:
                            if D.iloc[i,41] == 1:
                                color = 'pearl'
                            elif D.iloc[i,41] == -1:
                                color = 'black'
            elif D.iloc[i,38] == 0:
                if D.iloc[i,36] != 0:
                    if D.iloc[i,34] == D.iloc[i,36]:
                        if D.iloc[i,23] == 0:
                            if D.iloc[i,22] == 0:
                                if D.iloc[i,21] == 0:
                                    color = 'gray'
                    elif D.iloc[i,34] == 0:
                        if D.iloc[i,23] == 0:
                            if D.iloc[i,22] == 0:
                                if D.iloc[i,21] == 0:
                                    if D.iloc[i,41] == 1:
                                        color = 'blue'
                                    elif D.iloc[i,41] == -1:
                                        color = 'purple'
                                elif D.iloc[i,21] != 0:
                                    for ugo2 in goldpa2:
                                        if D.iloc[i,21] == ugo2:
                                            color = 'gold'
                                        elif D.iloc[i,21] == 0:
                                            if D.iloc[i,20] != 0:
                                                for ugo in goldpa:
                                                    if D.iloc[i,20] == ugo:
                                                        color = 'gold'
                    elif D.iloc[i,34] == 0:
                        if D.iloc[i,21] == 0:
                            for uy in ypa:
                                if D.iloc[i,20] == uy:
                                    if D.iloc[i,41] == 1:
                                        color = 'yellow'
                                    elif D.iloc[i,41] == -1:
                                        color = 'orange'
                            for ug in grpa:
                                if D.iloc[i,20] == ug:
                                    if D.iloc[i,41] == 1:
                                        color = 'green'
                                    elif D.iloc[i,41] == -1:
                                        color = 'red'
            COLOR.append(color)
            i = i + 1

        D['COLOR'] = COLOR


## NEW BUILD - MOMENTUM ANALYSIS USING 1ST AND 2ND DERIVATIVES



        Hrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H2, 0, 25)

        H = pd.DataFrame(Hrates)
        H['time']=pd.to_datetime(H['time'], unit='s')
        H = H.drop(columns=['tick_volume', 'spread', 'real_volume'])
        H = H.rename({'time': 'date', 'open': 'a', 'high': 'b', 'low': 'c', 'close': 'd'}, axis='columns')
        H = H.set_index('date')
        H = H.assign(e=H['a'])
        H = H.assign(f=H['b'])
        H = H.assign(g=H['c'])
        H = H.assign(h=H['d'])
        H = H.assign(tickqty=0)
        H = H.assign(open=H['a'])
        H = H.assign(close=H['d'])
        H = H.assign(high=H['b'])
        H = H.assign(low=H['c'])
        H = H.tz_localize('Etc/GMT-7')
        H = H.tz_convert('UTC')
        H = H.assign(MOVE=(H['close'] - H['open']))
        hl = int(H['close'].count())
        hlin = np.linspace(1,hl,num=hl)

        SLOPE = []
        i = 0
        for t in hlin:
            ii = i - 1
            if i == 0:
                SLOPE.append(0)
            else:
                slope = H.iloc[i,10] - H.iloc[ii,10]
                SLOPE.append(slope)
            i = i + 1

        SLOPE2 = []
        i = 0
        for t in hlin:
            ii = i - 1
            if i == 0:
                SLOPE2.append(0)
            else:
                slope2 = SLOPE[i] - SLOPE[ii]
                SLOPE2.append(slope2)
            i = i + 1

        CONCAVE = []
        i = 0
        for t in hlin:
            if SLOPE2[i] == 0:
                CONCAVE.append('INFL')
            elif SLOPE2[i] > 0:
                CONCAVE.append('UP')
            else:
                CONCAVE.append('DOWN')
            i = i + 1

        axiom = D['close'].count()/H['close'].count()
        AXCONCAVE = []
        i = 1
        for t in lin:
            alpha = int(math.ceil(i/axiom)-1)
            axconcave = CONCAVE[alpha]
            AXCONCAVE.append(axconcave)
            i = i + 1
        D['CONCAVE'] = AXCONCAVE







## HERE IS WHERE WE WILL CODE THE BOT-SPECIFIC INTERPRETATION ALGORITHM
## THIS BOT IS NAMED MARVIN
## MARVIN IS DESIGNED TO USE COLOR-CODE TO IDENTIFY PRICE ACTION AS BREAK AND RETESTS
## HE IS ALSO DESIGNED TO USE CONCAVITY AND THE ICHIMOKU BLUE LINE AS CONFLUENCE GRADING TOOLS
## WE WILL FLAG EACH PAIR WITH A GOOD TRADE OPPORTUNITY


        COLTRADE = []
        i = 0
        for t in lin:
            io = i - 1
            if i == 0:
                trade = 0
            else:
                if D.iloc[i,42] == 'purple':
                    #if D.iloc[i,42] == 'pearl':
                    trade = 'TRADE'
                elif D.iloc[i,42] == 'gold':
                    #if D.iloc[i,42] == 'pearl':
                    trade = 'TRADE'
                else:
                    trade = 0
            COLTRADE.append(trade)
            i = i + 1
        D['COLTRADE'] = COLTRADE

        CONTRADE = []
        i = 0
        for t in lin:
            if D.iloc[i,44] == 'TRADE':
                if D.iloc[i,19] > 0:
                    if D.iloc[i,43] == 'UP':
                        contrade = 'TRADE'
                elif D.iloc[i,19] < 0:
                    if D.iloc[i,43] == 'DOWN':
                        contrade = 'TRADE'
            else:
                contrade = 0
            CONTRADE.append(contrade)
            i = i + 1
        D['CONTRADE'] = CONTRADE

        ICHTRADE = []
        BIGBLUE = []
        blulin = np.linspace(25,0,num=26)
        for t in blulin:
            ICHTRADE.append(0)
        i = 26
        ib = int(D['close'].count() - 26)
        blilin = np.linspace(1,ib,num=ib)
        for t in blilin:
            for h in blulin:
                ih = int(i - h)
                bigblue = D.iloc[ih,31]
                BIGBLUE.append(bigblue)
            trade = 'TRADE'
            for j in BIGBLUE:
                if D.iloc[i,19] > 0:
                    if j == -1:
                        trade = 0
                elif D.iloc[i,19] < 0:
                    if j == 1:
                        trade = 0
                else:
                    trade = 0
            ICHTRADE.append(trade)
            i = i + 1
            BIGBLUE = []
        D['ICHTRADE'] = ICHTRADE

        POSITION = []
        i = 0
        for t in lin:
            if D.iloc[i,44] == 'TRADE':
                if D.iloc[i,45] == 'TRADE':
                    if D.iloc[i,46] == 'TRADE':
                        if D.iloc[i,19] > 0:
                            position = 'LONG'
                        elif D.iloc[i,19] < 0:
                            position = 'SHORT'
            else:
                position = 0
            POSITION.append(position)
            i = i + 1
        D['POSITION'] = POSITION

        ENTRY = []
        i = 0
        for t in lin:
            if D.iloc[i,47] == 'LONG':
                entry = D.iloc[i,36] + CM
            elif D.iloc[i,47] == 'SHORT':
                entry = D.iloc[i,36] - CM
            else:
                entry = 0
            i = i + 1
            ENTRY.append(entry)
        D['ENTRY'] = ENTRY

## THIS BLOCK CHOOSES THE RESPECTIVE LEVELS FOR ENTRY, STOP LOSS, AND TAKE PROFIT        

        TAKEPROFIT = []
        STOPLOSS = []
        cend = int(len(levels)-1)
        i = 0
        for t in lin:
            if D.iloc[i,47] == 'LONG':
                clevel = D.iloc[i,36]
                clindex = levels.index(clevel)
                tpindex = int(clindex - 1)
                slindex = int(clindex + 1)
                if clindex == 0:
                    stoploss = levels[slindex]
                    sldist = clevel - stoploss
                    takeprofit = clevel + sldist
                elif clindex == cend:
                    takeprofit = levels[tpindex]
                    tpdist = takeprofit - clevel
                    stoploss = clevel - tpdist
                else:
                    takeprofit = levels[tpindex]
                    stoploss = levels[slindex]
            elif D.iloc[i,47] == 'SHORT':
                clevel = D.iloc[i,36]
                clindex = levels.index(clevel)
                tpindex = int(clindex + 1)
                slindex = int(clindex - 1)
                if clindex == 0:
                    takeprofit = levels[tpindex]
                    tpdist = clevel - takeprofit
                    stoploss = clevel + tpdist
                elif clindex == cend:
                    stoploss = levels[slindex]
                    sldist = stoploss - clevel
                    takeprofit = clevel - sldist
                else:
                    takeprofit = levels[tpindex]
                    stoploss = levels[slindex]
            else:
                takeprofit = 0
                stoploss = 0
            i = i + 1
            TAKEPROFIT.append(takeprofit)
            STOPLOSS.append(stoploss)
        D['TAKEPROFIT'] = TAKEPROFIT
        D['STOPLOSS'] = STOPLOSS

        shrink = 0.90
        i = 0
        for t in lin:
            if D.iloc[i,47] == 'LONG':
                entry = D.iloc[i,48]
                tpdist = D.iloc[i,49]-D.iloc[i,48]
                takeprofit = entry + tpdist*shrink
                D.iloc[i,49] = takeprofit
            elif D.iloc[i,47] == 'SHORT':
                entry = D.iloc[i,48]
                tpdist = D.iloc[i,48]-D.iloc[i,49]
                takeprofit = entry - tpdist*shrink
                D.iloc[i,49] = takeprofit
            i = i + 1


        i = 0
        for t in lin:
            if D.iloc[i,47] == 'LONG':
                entry = D.iloc[i,48]
                tpdist = D.iloc[i,49]-D.iloc[i,48]
                sldist = D.iloc[i,48]-D.iloc[i,50]
                if sldist > tpdist:
                    sldist = tpdist
                    stoploss = entry-sldist
                    D.iloc[i,50] = stoploss
            elif D.iloc[i,47] == 'SHORT':
                entry = D.iloc[i,48]
                tpdist = D.iloc[i,48]-D.iloc[i,49]
                sldist = D.iloc[i,50]-D.iloc[i,48]
                if sldist > tpdist:
                    sldist = tpdist
                    stoploss = entry+sldist
                    D.iloc[i,50] = stoploss
            i = i + 1




        D.drop(D.tail(1).index,inplace=True)
        ldc = int(D['close'].count())
        la = int(ldc - 1)
        if D.iloc[la,47] != 0:
            pairing = x
            colcode = D.iloc[la,42]
            position = D.iloc[la,47]
            entry = D.iloc[la,48]
            tkprofit = D.iloc[la,49]
            stloss = D.iloc[la,50]
            #playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/done.mp3')
            #playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/MARVIN2.mp3')
            df = pd.DataFrame(pairing,position,entry,tkprofit,stloss)
            purps.append(df)

        #TESTING AREA

        #ZA.append(x)
        #ZB.append(SAvgCandle)
        #ZC.append(DAvgCandle)

        #sheet = x
        #sheet = sheet.replace("/", "")


        #LEVELS.to_excel(writerdft, sheet_name=sheet)
        #D.index = D.index.strftime('%m %d %Y %H:%M:%S')
        #D.to_excel(writerdfd, sheet_name=sheet)

    #Output = zip(ZA,ZB,ZC)
    #Output = pd.DataFrame(Output)
    #winsound.PlaySound(None, winsound.SND_ASYNC)
    #print(Output)



    print(purps)
    playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/done.mp3')
    #playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/MARVIN2.mp3')


                

In [7]:
TIMER = []
iterator = 0

while iterator < 5:
    start = dt.now()
    brutus()
    end = dt.now()
    #trades = purps
    #print(trades)
    elapsed = end - start
    elapsed = elapsed.total_seconds()
    elapsed = round(elapsed/60,2)
    elapsed = str(elapsed)
    timer = elapsed + '   minutes'
    TIMER.append(timer)
    time.sleep(5)
    iterator = iterator + 1


print(TIMER)

Empty DataFrame
Columns: [pairing, position, entry, tkprofit, stloss]
Index: []
Empty DataFrame
Columns: [pairing, position, entry, tkprofit, stloss]
Index: []
Empty DataFrame
Columns: [pairing, position, entry, tkprofit, stloss]
Index: []
Empty DataFrame
Columns: [pairing, position, entry, tkprofit, stloss]
Index: []
Empty DataFrame
Columns: [pairing, position, entry, tkprofit, stloss]
Index: []
['4.99   minutes', '5.43   minutes', '5.48   minutes', '5.21   minutes', '5.13   minutes']


In [8]:
mt5.shutdown()

True